## Carreguem dlib detector

In [ ]:
import itertools
import kagglehub
import os
import cv2
import numpy as np
import glob
import shutil
import pandas as pd
import dlib
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import zip_longest
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from skimage.feature import hog
from skimage import exposure
import cv2
import os
import numpy as np
import mediapipe as mp
import pywt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import kagglehub
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.utils import to_categorical
import warnings; warnings.simplefilter('ignore')

In [ ]:
dlib_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("Predictors\shape_predictor_68_face_landmarks.dat")

## Detectem cara dlib

In [ ]:
def extract_mouth_dlib(img_gray):
    faces = dlib_detector(img_gray)
    if len(faces) == 0:
        print("No face detected.")
        return None

    for face in faces:
        landmarks = landmark_predictor(img_gray, face)

        mouth_points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)]
        mouth_np = np.array(mouth_points)

        x, y, w, h = cv2.boundingRect(mouth_np)
        mouth_roi = img_gray[y:y+h, x:x+w]

        if mouth_roi.size == 0:
            print("Empty mouth.")
            return None

        try:
            resized = cv2.resize(mouth_roi, (64, 64))
        except Exception as e:
            print("Error:", e)
            return None

        features = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        return features

    return None


## Carrreguem Facial emotion Recognition

In [ ]:
source_dir = kagglehub.dataset_download("tapakah68/facial-emotion-recognition")
people_dirs = glob(os.path.join(source_dir+'/images', "*"))

# Path al dataset organitzat
dest_dir = "/kaggle/working/organized_by_emotion"
os.makedirs(dest_dir, exist_ok=True)

# Possibles emocions
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']

# Organitzem emocions per carpetes
for emotion in emotions:
    os.makedirs(os.path.join(dest_dir, emotion), exist_ok=True)


# Movem les emocions a les carpetes
for person_path in people_dirs:
    if os.path.isdir(person_path):
        person_id = os.path.basename(person_path)
        emotion_images = glob(os.path.join(person_path, "*.jpg"))

        for img_path in emotion_images:
            emotion = os.path.splitext(os.path.basename(img_path))[0].capitalize()
            if emotion in emotions:
                new_filename = f"{person_id}.jpg"
                dest_path = os.path.join(dest_dir, emotion, new_filename)
                shutil.copy(img_path, dest_path)

Train amb tres possibles models

In [ ]:
def train(model, file_path, emotions):
  X_train, y_train = [], []

  for emotion in emotions:
      emotion_path = os.path.join(file_path, emotion)
      files = sorted(os.listdir(emotion_path))[:12]  #Agafem les 12 pirmeres persones per fer el train

      for filename in files:
          img_path = os.path.join(emotion_path, filename)
          img = cv2.imread(img_path)
          if img is None:
              continue
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          features = extract_mouth_dlib(gray)

          if features is not None:
              X_train.append(features)
              y_train.append(emotion)

  # Encode les etiquetes
  le = LabelEncoder()
  y_train_enc = le.fit_transform(y_train)

  # Entrenem el model corresponent
  if model == 'rf':
      rf = RandomForestClassifier(n_estimators=100, random_state=42)
      rf.fit(X_train, y_train_enc)

  elif model == 'knn':
    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    knn.fit(X_train, y_train_enc)

  elif model == 'svm':
    clf = SVC(kernel='linear', probability=True)
    clf.fit(X_train, y_train_enc)

  else:
      raise ValueError("Model not available")

  #Guardem el model
  with open('emotion_rf.pkl', 'wb') as f:
      pickle.dump((rf, le), f)

  print(f"Model trained ({model}) on {len(X_train)} samples.")


Test amb tres possibles models

In [ ]:
def test(model, file_path, emotions):
  with open('emotion_rf.pkl', 'rb') as f:
    rf, le = pickle.load(f)

    y_true, y_pred = [], []

    for emotion in emotions:
        emotion_path = os.path.join(file_path, emotion)
        files = sorted(os.listdir(emotion_path))[12:18]  # Últimes 6 persones per testing

        for filename in files:
            img_path = os.path.join(emotion_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            features = extract_mouth_dlib(gray)

            if features is not None:
                if model == 'rf':
                    pred = rf.predict([features])[0]
                elif model == 'knn':
                    pred = knn.predict([features])[0]
                elif model == 'svm':
                    pred = clf.predict([features])[0]

                pred_label = le.inverse_transform([pred])[0]
                y_true.append(emotion)
                y_pred.append(pred_label)

    #Resultats
    print(f"\n Results with model {model}:\n")
    print(classification_report(y_true, y_pred, zero_division=0))

    #Matriu de confusió
    cm = confusion_matrix(y_true, y_pred, labels=emotions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=emotions, yticklabels=emotions)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{model} Confusion Matrix')
    plt.tight_layout()
    plt.show()

Prova (podem canviar al model que vulguem)

In [ ]:
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']
DATA_DIR = '/kaggle/working/organized_by_emotion'

#model pot ser: rf, svm, knn
test(model='rf', file_path=DATA_DIR, emotions=emotions)
train(model='rf', file_path=DATA_DIR, emotions=emotions)